In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv
data = pd.read_csv('/content/segment601-892.csv', encoding='unicode_escape')

skills = pd.read_csv('/content/skills_data.csv',  encoding='latin')
SKILLS_DB = list(set(skills['Skills'].values))

In [ ]:
data

,cv,Information,Skill,Education,Experience,SU tag,O tag
0,601,"phone 84 979 398 849 , email tulangphieu gm...",pressure environmentiwant to build expertise ...,"education, bach khoa university vietnam natio...","tu lang phieu, temporary, may 12th 1979 , 9 ...","career objective, to establishasuccessful care...",phuoc hai commune ninh phuoc district ninh t...
1,602,"name , pham tuan son, mobile ...",good communication and team work also work wel...,"2009 2010 , 2007 2008 , education, 2005 2...","experience summary, skillful in business analy...",10 years of experience in software development...,tcp udp dns ftp sctp telnet http dhcp ...
2,603,pson2900 gmail com 84 1699805901 91 40 qua...,"ph?m nguy?n ng?c s?n, i have use git command ...",buildasimple neural networks with3dense layers...,"experience, learn and use raspberry pi butim s...",using pythonspickle using matplotlib to rende...,"brief information, imstillastudent learning at..."
3,604,"ngo thai ngoc cv confidential, personal deta...","databases etc my sql, programming tools visu...","educational background, university of informat...","0987096206, award, software, programming langu...","professional summary, cooperative and able to ...","curriculum vitae, cam son commune , complete c..."
4,605,"qualifications, nguyen thi anh thi, nguyenanht...",good at sql queries good knowledge in object ...,"education, 9 2011, studying in management info...",", all tasks on major which related to data ana...",to be always willing to learn more and make ev...,"orientation, complete workshop sap erp about p..."
...,...,...,...,...,...,...,...
131,882,"khoa nguyen, nguyenkhoa2407 gmail com, mobile ...",skills proficiency inc programming experie...,"education, vanderbilt university nashville t...","majors computer science chemistry, saint jos...",collect data on the degree of swelling compre...,"overall gpa374, honors and awards deanslist ..."
132,883,"personal information, email phucvu4697 gmail ...","technical skills, python matlabcc , other sk...","education, 2015 2019 university of science, s...","math machine learning, algorithms, projects, ...",apply for ai machine learning internship,"vu hoang phuc, ebolavirus classification, 2014"
133,885,"84 983521482, vietnam , phone 84 912 75...","skills, technical python javac c javas...",msc enterprise digital content engineering r...,"coulibaly, adama, coulibalysadama gmail com c...",virtual and augmented reality unity blende...,"curriculum vitae, academic quali?cations, 2017..."
134,889,"dob9january 1998, 84 389 552 357 nggbaobkit...","strengths, responsible always ?nish deadlines...","education, thoai ngoc hau high school for the ...","gia bao nguyen, september 2016, september 2017...",good leader ship skills experience of 14 year...,"volunteer experience, 2016 2017, humanitaria..."


In [ ]:
import nltk

def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"', " ", str(input_text))
    
    word_tokens = nltk.tokenize.word_tokenize(word_tokens)
    
    
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # create a set to keep the results in.
    found_skills = set()

    # search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

In [ ]:
#Extract skill from column Skill

df_skill = pd.DataFrame(columns=['skill', 'extract_skill'])
for i, j in zip(data['Skill'], data['Experience']):
    skills1 = extract_skills(i)
    skills2 = extract_skills(j)
    
    z = str(i) + ''.join(str(j))
    join_skills = skills1.union(skills2)
    df_skill = df_skill.append({'skill': z, 'extract_skill': join_skills}, ignore_index=True)

In [ ]:
#Extract degree and major from column Education using regex

edu_pattern = re.compile('college|master|bachelor|bachelors|.*major.*|.*speciality.*')

df_edu = pd.DataFrame(columns=['education', 'extract_edu'])

for i in data['Education']:
    lst_edu = []
    lst = str(i).split(',')

    for j in lst:
        edu = edu_pattern.findall(j)
        if bool(edu) == True:
            lst_edu.extend(edu)
        
    df_edu = df_edu.append({'education': i, 'extract_edu': lst_edu}, ignore_index=True)

In [ ]:
# ([2]{1}[0]{1}[0-9]{2})\s+[0-9]{2}\b
# \b[a-z]{3}\s+ 
# ([2]{1}[0]{1}[0-9]{2})
# ([2]{1}[0]{1}[0-9]{2})

year_pattern = re.compile('([2]{1}[0]{1}[0-9]{2})')
exp_pattern = re.compile('([0-9]{2})\s+year')

df_exp = pd.DataFrame(columns=['experience', 'extract_exp'])

for i, j in zip(data['Experience'], data['SU tag']):
    lst_exp = []
    text = str(i) + ' ' + str(j)
    exp = exp_pattern.findall(text)
    years = year_pattern.findall(text)
    
    if bool(exp)==True:
        to_int = [int(i) for i in exp]
        extract_year = exp
        n_exp = max(to_int)
    else:
        to_int = [int(i) for i in years]
        extract_year = years
        if len(to_int) > 0:
            max_exp = max(to_int)
            min_exp = min(to_int)
            n_exp = max_exp - min_exp
        else:
            n_exp = 'None'
            
    lst_exp.append(str(n_exp))
    
    df_exp = df_exp.append({'experience': text, 'extract_exp': lst_exp}, ignore_index=True)

In [ ]:
extracted_data = pd.DataFrame({'Skill': df_skill['skill'],
                              'Extract_Skill': df_skill['extract_skill'],
                               'Education': df_edu['education'],
                               'Extract_Edu': df_edu['extract_edu'],
                              'Experience': df_exp['experience'],
                              'Extract_Exp': df_exp['extract_exp'],
                              })

In [ ]:
extracted_data[1:50]

,Skill,Extract_Skill,Education,Extract_Edu,Experience,Extract_Exp
1,good communication and team work also work wel...,"{php, cvs, team work, data structures, access,...","2009 2010 , 2007 2008 , education, 2005 2...","[master, bachelor]","experience summary, skillful in business analy...",[10]
2,"ph?m nguy?n ng?c s?n, i have use git command ...","{s, tensorflow, machine learning, com, python,...",buildasimple neural networks with3dense layers...,[],"experience, learn and use raspberry pi butim s...",[None]
3,"databases etc my sql, programming tools visu...","{data processing, unit testing, proteus, atmel...","educational background, university of informat...",[],"0987096206, award, software, programming langu...",[2]
4,good at sql queries good knowledge in object ...,"{ajax, relational databases, research, sap erp...","education, 9 2011, studying in management info...",[],", all tasks on major which related to data ana...",[2]
5,"skills, soft skills, team work , presentation ...","{c, team work, presentation, network, python, ...","0978 928 781, education, 2011 2016 ho chi m...",[ 2011 2016 ho chi minh city university of ...,"cc , c , 06 2015 10 2015 joined in ccna cisc...",[0]
6,programming skill and an interest to advanced...,"{java, teamwork, python, chinese, problem solv...",korea advanced university of science and techn...,[],address areum hall kaist 291 daehak ro gu...,[None]
7,"google net, computer vision machine learning...","{ubuntu, java, machine learning, history, rese...","noi viet nam, education , graduate at ha noi ...",[ major ],"the basic information , nguyen dai phong, ...",[1]
8,"key skills, solid understanding of analytical ...","{research, economics, teamwork, analytics, log...","education, graduate at university of economics...",[bachelor],"system, gpa72 10 , text mining on the educat...",[6]
9,"technical skills, web server apache iis , go...","{statistics, network programming, linux, erp, ...","education, 2007 2010, 2010 2015, university of...",[],"viet nam, to acquire the position ofajunior n...",[None]
10,"programmer with4years experience , quickly gra...","{crystal reports, ajax, research, wpf, subvers...","education, graduated from institute of applie...",[],"1, from october 2008 to 2011 , industry infor...",[30]
